<p style="font-weight:bold;"> <span style="font-size: 36px"> Ifrs Variable </span> </p>

In [0]:
#!eval-notebook "Data/InitSystemorphToMemoryForTesting"

# BenchMarks

In [0]:
var pathToBm = "./Data/IfrsVariableBenchmarks/";

In [0]:
public record BenchmarkMetadata(string FileName, string ReportingNode, int Year, int Month, string Scenario = null){} 

In [0]:
var excludedDataNodes = new string[]{};
var includedDataNodes = new string[]{};

# Functions

In [0]:
public bool IsCorrectValue(double value) => Math.Abs(value) <= BenchmarkPrecision;

In [0]:
public static NumberStyles AllowedNumberStyles = NumberStyles.Float | NumberStyles.AllowLeadingWhite | NumberStyles.AllowTrailingWhite | NumberStyles.AllowLeadingSign | NumberStyles.AllowTrailingSign | NumberStyles.AllowDecimalPoint;

In [0]:
public double ParseIfrsVariableValue(IDataRow row, string nameOfIfrsVariableValue)
{
    return (row.Table.Columns.Contains(nameOfIfrsVariableValue)
                        ? (double.TryParse(row[nameOfIfrsVariableValue]?.ToString(), AllowedNumberStyles, CultureInfo.InvariantCulture, out var value) 
                                ? value 
                                : throw new Exception(nameOfIfrsVariableValue + " cannot be parsed."))
                        : default(double));
}

In [0]:
public IfrsVariable CreateVariable(IDataRow row, Guid partition)
    => new IfrsVariable(){AocType = row.Field<string>(nameof(IfrsVariable.AocType)),
                          Partition = partition,
                          Novelty = row.Field<string>(nameof(IfrsVariable.Novelty)),
                          DataNode = row.Field<string>(nameof(IfrsVariable.DataNode)),
                          EstimateType = row.Field<string>(nameof(IfrsVariable.EstimateType)),
                          AmountType = row.Field<string>(nameof(IfrsVariable.AmountType)),
                          AccidentYear = Int32.TryParse((row.Field<string>(nameof(IfrsVariable.AccidentYear))), out var accidentYear) ? accidentYear : (int?)null,
                          EconomicBasis = row.Field<string>(nameof(IfrsVariable.EconomicBasis)),
                          Value = Enumerable.Range(0, 13)
                                            .Select(x => ParseIfrsVariableValue(row, x == 0 ? "Value" : "Value" + x.ToString()))
                                            .ToArray();
                        /*
                          Value = new double[] {(-1.0) * ParseIfrsVariableValue(row, nameof(IfrsVariable.Value)), 
                                            (-1.0) * ParseIfrsVariableValue(row, "Value1"), 
                                            (-1.0) * ParseIfrsVariableValue(row, "Value2"),
                                            (-1.0) * ParseIfrsVariableValue(row, "Value3"), 
                                            (-1.0) * ParseIfrsVariableValue(row, "Value4"), 
                                            (-1.0) * ParseIfrsVariableValue(row, "Value5"),                          
                                            (-1.0) * ParseIfrsVariableValue(row, "Value6"), 
                                            (-1.0) * ParseIfrsVariableValue(row, "Value7"), 
                                            (-1.0) * ParseIfrsVariableValue(row, "Value8"),
                                            (-1.0) * ParseIfrsVariableValue(row, "Value9"), 
                                            (-1.0) * ParseIfrsVariableValue(row, "Value10"), 
                                            (-1.0) * ParseIfrsVariableValue(row, "Value11"),                          
                                            (-1.0) * ParseIfrsVariableValue(row, "Value12")} */
    };

In [0]:
public class BenchmarkTestResult
{
    public string ErrorMessage {get; set;}
    public double[] ExpectedValue {get; set;}
    public double[] ComputedValue {get; set;}
    public BenchmarkTestResult (string message)
    {
        ErrorMessage = message;
    }
    public BenchmarkTestResult (string message, double[] expValue, double[] compValue)
    {
        ErrorMessage = message;
        ExpectedValue = expValue;
        ComputedValue = compValue;    
    }
}

In [0]:
var includingProperties = typeof(IfrsVariable).GetIdentityProperties().Select(x=>x.Name).ToArray();

In [0]:
/*using System.Reflection;

public List<BenchmarkTestResult> CompareIfrsVariables(IfrsVariable expectedVariable, IfrsVariable computedVariable){
    var ret = new List<BenchmarkTestResult>();
    if (expectedVariable.Value == null) ret.Add(new BenchmarkTestResult("The expected Ifrs variable contains no BE or Projection values"));
    if (computedVariable.Value == null) ret.Add(new BenchmarkTestResult("The computed Ifrs variable contains no BE or Projection values"));
    if (ret.Any()) return ret;
    if (expectedVariable.Value.Count() != computedVariable.Value.Count()){
        ret.Add(new BenchmarkTestResult("The computed variable has " + computedVariable.Value.Count().ToString() + 
                                        " projections, while the expected has " + expectedVariable.Value.Count().ToString() +
                                        " entries, cannot compare", "", -1, -1));
        return ret;
    }
    return expectedVariable.Value.Select((x, i) => Math.Abs(x + computedVariable.Value.ElementAt(i)) > BenchmarkPrecision ?
                                                        new BenchmarkTestResult("Values do not match. Partition " + expectedVariable.Partition + 
                                                                                ", " + expectedVariable.ToIdentityString(), "Value" + i.ToString(), 
                                                                                -x, computedVariable.Value.ElementAt(i)) :
                                                        null )
                                        .Where(x => x != null).ToList(); */
}

In [0]:
public List<BenchmarkTestResult> CompareAgainstBm(List<BenchmarkTestResult> errors, IEnumerable<IfrsVariable> expected, IEnumerable<IfrsVariable> computed){
    var comparerNoValue = IfrsVariableComparer.Instance();
    var comparerWithValue = IfrsVariableComparer.Instance(true, BenchmarkPrecision);
    var expectedNotComputed = expected.Where(x => x.Value.Any(y => Math.Abs(y) > BenchmarkPrecision)).Except(computed, comparerNoValue);
    var computedNotExpected = computed.Where(x => x.Value.Any(y => Math.Abs(y) > BenchmarkPrecision)).Except(expected, comparerNoValue);
    var errors = new List<BenchmarkTestResult>(); 
    if (expectedNotComputed.Any()){
        foreach(var element in expectedNotComputed){
            errors.Add(new BenchmarkTestResult("Extra expected variable for: Partition " + element.Partition + ", " + element.ToIdentityString()));
        }
    }
    if (computedNotExpected.Any()){
        foreach(var element in computedNotExpected){
            errors.Add(new BenchmarkTestResult("Extra expected variable for: Partition " + element.Partition  + ", " + element.ToIdentityString()));
        }
    }
    foreach(var benchmarkVariable in expected){
        var computedVariable = computed.FirstOrDefault(x => comparer.Equals(benchmarkVariable, x)) ?? null;
        if (computedVariable == null) continue;
        if (!comparerWithValue.Equals(benchmarkVariable, computedVariable)) 
            errors.Add(new BenchmarkTestResult("Values do not match. Partition " + benchmarkVariable.Partition + ", " + benchmarkVariable.ToIdentityString(), 
                                                    benchmarkVariable.Variable, computedVariable.Variable))
        /*var newErrors = CompareIfrsVariables(benchmarkVariable, computedVariable);
        if (newErrors.Any()) errors = errors.Concat(newErrors).ToList();  */
    }
    return errors;
}

# Test Runner

In [0]:
public bool ValidateMainTab(IDataRow mainRow, string reportingNode, int year, int month)
{
    if (!int.TryParse(mainRow[nameof(PartitionByReportingNodeAndPeriod.Year)]?.ToString(), NumberStyles.AllowLeadingWhite | NumberStyles.AllowTrailingWhite , CultureInfo.InvariantCulture, out var mainYear))
            throw new Exception("Value Year cannot be parsed."); 
    if (!int.TryParse(mainRow[nameof(PartitionByReportingNodeAndPeriod.Month)]?.ToString(), NumberStyles.AllowLeadingWhite | NumberStyles.AllowTrailingWhite , CultureInfo.InvariantCulture, out var mainMonth))
            throw new Exception("Value Month cannot be parsed."); 
    var mainReportingNode = mainRow[nameof(PartitionByReportingNodeAndPeriod.ReportingNode)].ToString();
    
    return reportingNode == mainReportingNode && year == mainYear && month == mainMonth;
}

In [0]:
public async Task<List<BenchmarkTestResult>> CheckIfrsVariablesValuesAsync(BenchmarkMetadata[] bmFiles, string[] excludedDataNodes, string[] includedDataNodes)
{
    var errorList = new List<BenchmarkTestResult>();
    foreach (var g in bmFiles.GroupBy(x => (ReportingNode: x.ReportingNode, Year: x.Year, Month: x.Month, Scenario : x.Scenario)))
    {
        var reportingNode = g.Key.ReportingNode;
        var year = g.Key.Year;
        var month = g.Key.Month;
        var scenario = g.Key.Scenario;
    
        await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new {ReportingNode = reportingNode, Year = year, Month = month, Scenario = scenario});
        var partition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().ToArrayAsync())
                            .SingleOrDefault(x => x.ReportingNode == reportingNode && x.Year == year && x.Month == month && x.Scenario == scenario)?.Id ?? 
                            throw new Exception("Partition not found.");
    
        var computedVariablesByEstimateType = (await DataSource.Query<IfrsVariable>()
                                                            .Where(v => excludedDataNodes != null 
                                                                            ? !excludedDataNodes.Contains(v.DataNode) 
                                                                            : true)
                                                            .Where(v => includedDataNodes != null 
                                                                            ? includedDataNodes.Contains(v.DataNode) 
                                                                            : true)
                                                            .ToArrayAsync())
                                                            .ToDictionaryGrouped(x => x.EstimateType, x => x.ToArray());

        foreach (var bmfile in g)
        {
            var extension = System.IO.Path.GetExtension(bmfile.FileName);
            var stream = await Project.FileStorage.ReadAsync(bmfile.FileName);
            //Read Bm csv
            var bmDataTable = (await DataSetReader.ReadFromStream(stream).WithContentType(extension).ExecuteAsync()).DataSet;

            var mainTab = bmDataTable.Tables[Main].Rows.First();
            if(!ValidateMainTab(mainTab, reportingNode, year, month))
                throw new Exception("Metadata and main tab of the Benchmark are different.");

            var bmTableName = bmDataTable.Tables.Single(x => x.TableName != Main).TableName;
            var bmVariables = bmDataTable.Tables[bmTableName].Rows.Select(x => CreateVariable(x, partition));

            //Get the computed Data
            var computedVariables = computedVariablesByEstimateType.TryGetValue(bmTableName, out var ret) ? ret : Enumerable.Empty<IfrsVariable>();

            if(!computedVariables.Any() && bmVariables.Any())
                errorList.Add(new BenchmarkTestResult("No variables are computed for EstimateType: " + bmTableName, 0, 0));
             //CompareAgainstBm
            if (bmVariables.Any() && computedVariables.Any()) //TODO we are adding duplicates here
                errorList = errorList.Concat(CompareAgainstBm(errorList, bmVariables, computedVariables)).Distinct().ToList();
        }
    }
    return errorList;
}

## Value Test

### BenchMark to test Value0

In [0]:
var bmFiles = new BenchmarkMetadata[]{
    //2020 Q4
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__A.csv"   , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__AA.csv"  , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__APA.csv" , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__BE.csv"  , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__BEPA.csv", "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__C.csv"   , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__DA.csv"  , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__F.csv"   , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__OA.csv"  , "CH", 2020, 12),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12__RA.csv"  , "CH", 2020, 12),
    // 2021 Q1
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__A.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__AA.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__APA.csv"  , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__BE.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__BEPA.csv" , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__C.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__DA.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__F.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__L.csv"    , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__LR.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__OA.csv"   , "CH", 2021, 3),
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__RA.csv"   , "CH", 2021, 3),
    // 2020 Q4
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_BE.csv"   , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_BEPA.csv" , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_A.csv"    , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_APA.csv"  , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_DA.csv"   , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_C.csv"    , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_F.csv"    , "CH", 2020, 12, "MTUP10pct"),
    new BenchmarkMetadata(pathToBm+"BM_CH_2020_12_MTUP10pct_RA.csv"   , "CH", 2020, 12, "MTUP10pct"),
};

### Test

In [0]:
excludedDataNodes = new string[]{"DTP1.1"};
includedDataNodes = null;
var errorList = await CheckIfrsVariablesValuesAsync(bmFiles, excludedDataNodes, includedDataNodes);

In [0]:
errorList.Count().Should().Be(0);

## Projection Test

### BenchMark to test Projections

In [0]:
var bmFilesTestProjection = new BenchmarkMetadata[]{
    // 2021 Q1
    new BenchmarkMetadata(pathToBm+"BM_CH_2021_3__BE_Projection_DTP.csv"   , "CH", 2021, 3)
};

### Test

In [0]:
ProjectionCount = 5;
await Import.FromFile("../Files/TransactionalData/NominalCashflows_CH_2020_12_Projection.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync();
await Import.FromFile("../Files/TransactionalData/NominalCashflows_CH_2021_3_Projection.csv")
            .WithFormat(ImportFormats.Cashflow)
            .WithTarget(DataSource)
            .WithActivityLog()
            .ExecuteAsync();

In [0]:
excludedDataNodes = null;
includedDataNodes = new string[]{"DTP1.1"};
var projectionErrorList = await CheckIfrsVariablesValuesAsync(bmFilesTestProjection, excludedDataNodes, includedDataNodes);

In [0]:
projectionErrorList.Count().Should().Be(0);

### Reset ProjectionCount

In [0]:
ProjectionCount = 1;